<a href="https://colab.research.google.com/github/nmatsumoto-lgtm/study-KIKAGAKU/blob/main/%E4%B8%8D%E5%8B%95%E7%94%A3%E4%BE%A1%E6%A0%BC%E4%BA%88%E6%B8%AC%E3%82%A2%E3%83%97%E3%83%AA_Ver_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install lightgbm==4.5.0 optuna==3.6.1 joblib pandas numpy scikit-learn

In [ ]:
from __future__ import annotations
import json
from pathlib import Path
from dataclasses import dataclass


import numpy as np
import pandas as pd
import optuna
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import mean_squared_log_error
from joblib import dump

In [ ]:
SEED = 0
np.random.seed(SEED)

In [ ]:
IN = Path("data/interim/train_tabular.csv")
OUT_DIR = Path("models"); OUT_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
# ============================
# 1) データ読み込み & 追加特徴量
# ============================

def _oof_target_median(df: pd.DataFrame, key, y: np.ndarray, n_splits: int = 5) -> np.ndarray:
    """key は str でも list[str] でもOK。OOFで groupby(key) の中央値を割り当て。"""
    if isinstance(key, str):
        key_cols = [key]
    else:
        key_cols = list(key)
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    oof = np.zeros(len(df), dtype=float)
    key_df = df[key_cols].copy()
    for tr_idx, va_idx in kf.split(df):
        g = pd.DataFrame({"y": y[tr_idx]})
        for c in key_cols:
            g[c] = key_df.iloc[tr_idx][c].values
        med = g.groupby(key_cols)["y"].median()
        # map 用の結合キー（高速・簡易）
        tr_key = key_df.iloc[va_idx].astype(str).agg("§".join, axis=1)
        med_key = med.reset_index()
        med_key["__k__"] = med_key[key_cols].astype(str).agg("§".join, axis=1)
        map_dict = pd.Series(med_key["y"].values, index=med_key["__k__"]).to_dict()
        mapped = tr_key.map(map_dict)
        oof[va_idx] = mapped.fillna(float(np.median(y[tr_idx]))).values
    return oof

def build_table_and_features() -> tuple[pd.DataFrame, list[str], list[str], list[str], list[int]]:
    df = pd.read_csv(IN)

    # ===== 目標（対数）=====
    y = df["price_yen"].astype(float).values
    y_log = np.log1p(y)

    # ===== 追加特徴量 =====
    df["access_score"] = 1.0 / (1.0 + df["walk_min"].astype(float))
    df["sqrt_area"] = np.sqrt(np.clip(df["area_sqm"].astype(float), 0, None))
    df["log_area"] = np.log1p(np.clip(df["area_sqm"].astype(float), 0, None))
    df["age_sqrt"] = np.sqrt(np.clip(df["築年数"].astype(float), 0, None))
    df["area_x_access"] = df["area_sqm"].astype(float) * df["access_score"].astype(float)

    # 取引年（欠損は全体中央値で補完）
    if "txn_year" in df.columns:
        df["txn_year"] = pd.to_numeric(df["txn_year"], errors="coerce")
        df["txn_year"] = df["txn_year"].fillna(df["txn_year"].median())
    else:
        # 取引年が無いデータでも動くように、現在年で埋める
        df["txn_year"] = datetime.now().year

    # 年の離散化（木が扱いやすい整数カテゴリ）
    df["txn_year_int"] = df["txn_year"].round().astype(int)

    # 頻度特徴量
    station_cnt = df["station"].value_counts()
    layout_cnt  = df["layout"].value_counts()
    df["station_count"] = df["station"].map(station_cnt).fillna(0).astype(int)
    df["layout_count"]  = df["layout"].map(layout_cnt).fillna(0).astype(int)

    # OOF 目標エンコード（駅/間取り/駅×年） ※対数ターゲットで作る
    df["station_oof_median_log"]      = _oof_target_median(df, "station", y_log)
    df["layout_oof_median_log"]       = _oof_target_median(df, "layout",  y_log)
    df["station_year_oof_median_log"] = _oof_target_median(df, ["station", "txn_year_int"], y_log)

    # 推論時に使う学習時統計
    station_med_map = pd.DataFrame({"station": df["station"], "y_log": y_log}).groupby("station")["y_log"].median().to_dict()
    layout_med_map  = pd.DataFrame({"layout":  df["layout"],  "y_log": y_log}).groupby("layout")["y_log"].median().to_dict()
    st_year_med_map = (pd.DataFrame({"station": df["station"], "year": df["txn_year_int"], "y_log": y_log})
                       .groupby(["station","year"])["y_log"].median().to_dict())
    global_station_med = float(np.median(list(station_med_map.values()))) if len(station_med_map) else float(np.median(y_log))
    global_layout_med  = float(np.median(list(layout_med_map.values())))  if len(layout_med_map)  else float(np.median(y_log))

    # カテゴリ列
    feat_cat = ["station", "layout", "txn_year_int"]  # 年もカテゴリとして持たせる

    # 数値列（順序固定）
    feat_num_base = ["walk_min", "築年数", "area_sqm"]
    feat_num_extra = [
        "access_score", "sqrt_area", "log_area", "age_sqrt",
        "area_x_access", "station_count", "layout_count",
        "station_oof_median_log", "layout_oof_median_log",
        "station_year_oof_median_log",
    ]
    feat_num_all = feat_num_base + feat_num_extra

    # 単調性制約（[cat..., num...] の順）
    n_cat = len(feat_cat)
    # walk_min は負、築年数は負、area_sqm は正、それ以外 0
    mono_num = [-1, -1, +1] + [0] * (len(feat_num_all) - 3)
    monotone_constraints = [0] * n_cat + mono_num

    # ====== エンコードして学習用テーブル ======
    oe = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
    X_cat = oe.fit_transform(df[feat_cat].fillna("NA")).astype("int32")
    X_num = df[feat_num_all].astype(float).values
    X = np.hstack([X_cat, X_num])

    cat_cols = [f"cat__{c}" for c in feat_cat]
    num_cols = feat_num_all.copy()
    X_df = pd.DataFrame(X, columns=cat_cols + num_cols)
    categorical_feature = cat_cols

    meta = {
        "feat_cat": feat_cat,
        "feat_num": num_cols,
        "categorical_feature": categorical_feature,
        "monotone_constraints": monotone_constraints,
    }

    infer_maps = {
        "station_count_map": station_cnt.to_dict(),
        "layout_count_map": layout_cnt.to_dict(),
        "station_median_log_map": station_med_map,
        "layout_median_log_map": layout_med_map,
        "station_year_median_log_map": {f"{k[0]}§{k[1]}": float(v) for k, v in st_year_med_map.items()},
        "global_station_median_log": global_station_med,
        "global_layout_median_log": global_layout_med,
        "feat_cat": feat_cat,
        "feat_num": num_cols,
    }
    return X_df, y_log, categorical_feature, monotone_constraints, meta, oe, infer_maps


In [ ]:
# ============================
# 2) Optuna でハイパラ探索
# ============================

def tune_with_optuna(
    X_df: pd.DataFrame,
    y_log: np.ndarray,
    categorical_feature: list[str],
    monotone_constraints: list[int],
    n_trials: int = 80,
) -> tuple[dict, int, float]:
    kf = KFold(n_splits=5, shuffle=True, random_state=SEED)

    def objective(trial: optuna.Trial) -> float:
        params = {
            "objective": "rmse",
            "metric": "rmse",
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.08, log=True),
            "num_leaves": trial.suggest_int("num_leaves", 31, 127),                 # ← 上限を抑制
            "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 120),     # ← 下限を底上げ
            "feature_fraction": trial.suggest_float("feature_fraction", 0.6, 0.9),
            "bagging_fraction": trial.suggest_float("bagging_fraction", 0.7, 1.0),
            "bagging_freq": trial.suggest_int("bagging_freq", 1, 10),               # ← 1以上を強制
            "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0.0, 3.0),
            "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 1.0, log=True),
            "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 1.0, log=True),
            "verbosity": -1,
            "seed": SEED,
            "monotone_constraints": monotone_constraints,
        }
        rmsles, best_iters = [], []
        for tr_idx, va_idx in kf.split(X_df):
            X_tr, X_va = X_df.iloc[tr_idx], X_df.iloc[va_idx]
            y_tr, y_va = y_log[tr_idx], y_log[va_idx]
            dtr = lgb.Dataset(X_tr, label=y_tr, categorical_feature=categorical_feature, free_raw_data=True)
            dva = lgb.Dataset(X_va, label=y_va, categorical_feature=categorical_feature, free_raw_data=True)

            model = lgb.train(
                params,
                dtr,
                valid_sets=[dva],
                num_boost_round=4000,
                callbacks=[lgb.early_stopping(300, verbose=False)],  # ← 少し長めに
            )
            pred_log = model.predict(X_va, num_iteration=model.best_iteration)
            rmsle = float(np.sqrt(mean_squared_log_error(np.expm1(y_va), np.expm1(pred_log))))
            rmsles.append(rmsle)
            best_iters.append(model.best_iteration)
        trial.set_user_attr("best_iters", best_iters)
        return float(np.mean(rmsles))

    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=n_trials)

    best_params = study.best_params
    best_iters_list = study.best_trial.user_attrs.get("best_iters", [1000])
    best_round = int(np.clip(np.mean(best_iters_list), 100, 4000))
    best_score = study.best_value
    return best_params, best_round, best_score

In [ ]:
# ============================
# 3) 最終学習（ポイント & 分位点）
# ============================

def train_final_models(
    X_df: pd.DataFrame,
    y_log: np.ndarray,
    categorical_feature: list[str],
    monotone_constraints: list[int],
    best_params: dict,
    best_round: int,
):
    base_params = dict(best_params)
    base_params.update({
        "objective": "rmse",
        "metric": "rmse",
        "verbosity": -1,
        "seed": SEED,
        # ★ ポイントモデルでは単調性制約を使う
        "monotone_constraints": monotone_constraints,
    })

    # 生データを解放しない（categorical_feature を後で使えるように）
    dtrain = lgb.Dataset(
        X_df,
        label=y_log,
        categorical_feature=categorical_feature,
        free_raw_data=False
    )

    # ---- ポイント（対数価格）----
    model_point = lgb.train(base_params, dtrain, num_boost_round=best_round)

    # ---- 分位点（q10/q90）----
    models_q = {}
    for alpha, tag in [(0.1, "q10"), (0.9, "q90")]:
        q_params = dict(best_params)
        q_params.update({
            "objective": "quantile",
            "alpha": alpha,
            "metric": "quantile",
            "verbosity": -1,
            "seed": SEED,
            # ★ ここが重要：quantile では単調性制約を外す
            # "monotone_constraints": monotone_constraints,  # ←入れない
        })
        models_q[tag] = lgb.train(q_params, dtrain, num_boost_round=best_round)

    return model_point, models_q


In [ ]:
# ===== 入力CSV→中間CSVの自動生成（なければ作る） =====
from pathlib import Path
import pandas as pd, numpy as np, re
from datetime import datetime

RAW_CSV = Path("/content/Tokyo_Chuo Ward_20222_20251.csv")
IN = Path("data/interim/train_tabular.csv")
ST_OUT = Path("data/interim/stations.csv")
IN.parent.mkdir(parents=True, exist_ok=True)

def _read_chuo_and_save(raw_csv: Path, out_csv: Path, stations_csv: Path):
    # エンコーディングを順に試す（Excel想定）
    last_err = None
    for enc in ("cp932", "utf-8-sig", "utf-8", "utf-16", "utf-16le", "utf-16be"):
        try:
            raw = pd.read_csv(raw_csv, encoding=enc)
            break
        except Exception as e:
            last_err = e
            raw = None
    if raw is None:
        raise last_err

    # ---- 年の抽出（和暦/西暦対応）----
    def _to_year(x):
        if pd.isna(x): return np.nan
        s = str(x)
        m = re.search(r"(\d{4})年", s)
        if m: return int(m.group(1))
        era_base = {"令和":2018, "平成":1988, "昭和":1925}
        for era, base in era_base.items():
            m = re.search(fr"{era}\s*(\d+)年", s)
            if m: return base + int(m.group(1))
        return np.nan

    # ---- 取引年（列が無ければ建物の保存年やファイル名からフォールバック）----
    def _to_txn_year(s):
        if pd.isna(s): return np.nan
        ss = str(s)
        # 例: "2024年第2四半期", "2023年上半期" などから西暦抽出
        m = re.search(r"(\d{4})", ss)
        return float(m.group(1)) if m else np.nan

    # 欲しい列が増えても落ちないように存在チェック
    need_base = ["最寄駅：名称","最寄駅：距離（分）","間取り","面積（㎡）","建築年","取引価格（総額）"]
    miss = [c for c in need_base if c not in raw.columns]
    if miss:
        raise KeyError(f"必要列が見つかりません: {miss}")
    df = raw[need_base].copy()

    # 取引時期があれば読む
    txn_year = None
    if "取引時期" in raw.columns:
        txn_year = raw["取引時期"].apply(_to_txn_year)
    # 全欠損なら建築年からの近似（現行の現在年は避ける）
    if txn_year is None or txn_year.isna().all():
        # ファイル名に西暦があれば拾う（例: Tokyo_Chuo Ward_20242_20251.csv → 2024〜2025）
        mfile = re.findall(r"(20\d{2})", raw_csv.name)
        fallback_year = float(mfile[-1]) if mfile else np.nan
        txn_year = pd.Series(fallback_year, index=df.index, dtype="float64")

    # ---- 築年数 = 取引年 − 建築年 ----
    build_year = df["建築年"].apply(_to_year)
    df["txn_year"] = txn_year.astype(float)
    df["築年数"] = (df["txn_year"] - build_year).clip(lower=0)

    # ---- 数値化ユーティリティ ----
    def _to_num_series(s):
        s = s.astype(str).str.replace(r"[^\d\.]", "", regex=True)
        return pd.to_numeric(s, errors="coerce")

    # ---- 数値列の強制数値化 & 欠損補完 ----
    for col in ["最寄駅：距離（分）","面積（㎡）","築年数"]:
        s_num = _to_num_series(df[col])
        med = s_num.median()
        if pd.isna(med):
            defaults = {"最寄駅：距離（分）":10.0, "面積（㎡）":40.0, "築年数":20.0}
            med = defaults.get(col, 0.0)
        df[col] = s_num.fillna(med)

    # ---- 間取りの正規化 ----
    def _normalize_layout(s):
        if not isinstance(s,str): return "その他"
        s = s.upper().replace("Ｌ","L").replace("Ｄ","D").replace("Ｋ","K").replace("Ｒ","R")
        m = re.search(r"(\d+)(LDK|DK|K|R)", s)
        return m.group(0) if m else "その他"

    # ---- 学習用テーブル ----
    train = df.rename(columns={
        "最寄駅：名称":"station",
        "最寄駅：距離（分）":"walk_min",
        "面積（㎡）":"area_sqm",
        "取引価格（総額）":"price_yen",
    })[["station","walk_min","築年数","area_sqm","price_yen","txn_year"]].copy()
    train["layout"] = raw["間取り"].astype(str).apply(_normalize_layout)

    # 不適切値の除去とクリップ
    train["price_yen"] = _to_num_series(train["price_yen"]).fillna(0)
    train = train[
        (train["price_yen"] > 0) &
        (train["area_sqm"]  > 0) &
        (train["walk_min"]  >= 0) &
        (train["築年数"]      >= 0)
    ].copy()
    train["area_sqm"] = train["area_sqm"].clip(8, 300)
    train["walk_min"] = train["walk_min"].clip(0, 60)

    # 保存
    train.to_csv(out_csv, index=False)

    # 駅リスト
    s = (raw["最寄駅：名称"].astype(str)
           .replace({"nan":np.nan,"None":np.nan,"":np.nan})
           .dropna()
           .str.normalize("NFKC")
           .str.replace("（","(",regex=False).str.replace("）",")",regex=False)
           .str.replace("　"," ",regex=False).str.strip()
           .str.replace(r"\s+"," ",regex=True))
    pd.Series(sorted(s.unique())).to_csv(stations_csv, index=False, header=False, encoding="utf-8-sig")


# まだ中間CSVがない場合は作る
if not IN.exists():
    if not RAW_CSV.exists():
        raise FileNotFoundError(f"Raw CSV が見つかりません: {RAW_CSV.resolve()}")
    _read_chuo_and_save(RAW_CSV, IN, ST_OUT)

In [ ]:
# ============================
# 4) 実行
# ============================

if __name__ == "__main__":
    assert IN.exists(), f"Not found: {IN}"

    # ★ infer_maps まで受け取る（関数が infer_maps を返す実装になっている前提）
    X_df, y_log, cat_cols, mono_cons, meta, oe, infer_maps = build_table_and_features()

    print(f"Train rows: {len(X_df):,}, cols: {X_df.shape[1]} (cats={len(cat_cols)})")

    best_params, best_round, best_score = tune_with_optuna(
        X_df, y_log, cat_cols, mono_cons, n_trials=80
    )
    print("[OPTUNA] best RMSLE:", round(best_score, 4))
    print("[OPTUNA] best params:")
    for k, v in best_params.items():
        print(f"  {k}: {v}")
    print("[OPTUNA] best_round:", best_round)

    model_point, models_q = train_final_models(
        X_df, y_log, cat_cols, mono_cons, best_params, best_round
    )

    # ===== 保存 =====
    # 予測時に使うメタ情報（列順や制約）とエンコーダを一緒に保存しておく
    dump({
        "model": model_point,
        "encoder": oe,
        "cat_cols": meta["feat_cat"],
        "num_cols": meta["feat_num"],
        "categorical_feature": meta["categorical_feature"],
        "monotone_constraints": meta["monotone_constraints"],
        "target": "log_price_yen",
    }, OUT_DIR / "lgbm_optuna_point.pkl")

    dump({
        "model": models_q["q10"],
        "encoder": oe,
        "cat_cols": meta["feat_cat"],
        "num_cols": meta["feat_num"],
        "categorical_feature": meta["categorical_feature"],
        "monotone_constraints": meta["monotone_constraints"],
        "target": "log_price_yen",
        "alpha": 0.1,
    }, OUT_DIR / "lgbm_optuna_q10.pkl")

    dump({
        "model": models_q["q90"],
        "encoder": oe,
        "cat_cols": meta["feat_cat"],
        "num_cols": meta["feat_num"],
        "categorical_feature": meta["categorical_feature"],
        "monotone_constraints": meta["monotone_constraints"],
        "target": "log_price_yen",
        "alpha": 0.9,
    }, OUT_DIR / "lgbm_optuna_q90.pkl")

    with open(OUT_DIR / "feature_config.json", "w", encoding="utf-8") as f:
        json.dump({
            "feat_cat": meta["feat_cat"],
            "feat_num": meta["feat_num"],
            "categorical_feature": meta["categorical_feature"],
            "monotone_constraints": meta["monotone_constraints"],
            "notes": "y is log1p(price_yen). During inference, apply expm1 to predictions.",
        }, f, ensure_ascii=False, indent=2)

    # ★ 推論用マップも保存（カッコを閉じるのを忘れない）
    with open(OUT_DIR / "infer_maps.json", "w", encoding="utf-8") as f:
        json.dump(infer_maps, f, ensure_ascii=False, indent=2)

    print("Saved models →", OUT_DIR)


[I 2025-09-04 13:32:41,598] A new study created in memory with name: no-name-23a12e22-7307-483b-8549-2a334a516d27


Train rows: 5,333, cols: 16 (cats=3)


[I 2025-09-04 13:32:42,309] Trial 0 finished with value: 0.208833667647713 and parameters: {'learning_rate': 0.01051212384762145, 'num_leaves': 67, 'min_data_in_leaf': 34, 'feature_fraction': 0.7465186911231043, 'bagging_fraction': 0.7759636930827386, 'bagging_freq': 8, 'min_gain_to_split': 2.9940882529901307, 'lambda_l1': 4.559494182949361e-06, 'lambda_l2': 1.4604132743874646e-08}. Best is trial 0 with value: 0.208833667647713.
[I 2025-09-04 13:32:42,950] Trial 1 finished with value: 0.20567005252943066 and parameters: {'learning_rate': 0.04161378886114005, 'num_leaves': 61, 'min_data_in_leaf': 69, 'feature_fraction': 0.7213175651586841, 'bagging_fraction': 0.9373779572879641, 'bagging_freq': 1, 'min_gain_to_split': 2.7030469357311944, 'lambda_l1': 1.8339102816463442e-06, 'lambda_l2': 7.30934896233308e-08}. Best is trial 1 with value: 0.20567005252943066.
[I 2025-09-04 13:32:43,586] Trial 2 finished with value: 0.20764929523073125 and parameters: {'learning_rate': 0.05053664916177309,

[OPTUNA] best RMSLE: 0.1825
[OPTUNA] best params:
  learning_rate: 0.0643313404097897
  num_leaves: 53
  min_data_in_leaf: 70
  feature_fraction: 0.6972048201585458
  bagging_fraction: 0.9206658571484022
  bagging_freq: 1
  min_gain_to_split: 0.001996946188143104
  lambda_l1: 1.5863435468056973e-05
  lambda_l2: 0.005004053354475371
[OPTUNA] best_round: 333
Saved models → models
